In [ ]:
"""
New Approach for a Simple Bidirectional A* algorithm such that Bidirectional Dijkstra using modified length
This algorithm is same with bidirectional A* with consistent approach
"""

In [ ]:
import import_ipynb
import Graph
import numpy as np
import pandas as pd
import time
from math import inf, sin, cos, asin, acos, pi, radians
import heapq as hq

In [ ]:
graph = Graph.make_graph('./roadnetwork_data/Ulsan_nodes.csv', './roadnetwork_data/Ulsan_edges.csv')

In [ ]:
# calculate heuristic
def for_hueristic(frm, target):
    R = 6400 # km
    sigma = acos(sin(radians(frm.lat))*sin(radians(target.lat))
                 + cos(radians(frm.lat))*cos(radians(target.lat))*cos(radians(abs(frm.lon-target.lon))))
    heuristic = R*sigma
    return heuristic*1000
    
def back_heuristic(start, to):
    R = 6400 # km
    sigma = acos(sin(radians(start.lat))*sin(radians(to.lat))
                 + cos(radians(start.lat))*cos(radians(to.lat))*cos(radians(abs(start.lon-to.lon))))
    heuristic = R*sigma
    return heuristic*1000

def heuristic(frm, to, start, target):
    return (1/2)*(for_hueristic(to, target)-for_hueristic(frm, target)) + (1/2)*(back_heuristic(start, frm)-back_heuristic(start, to))

In [ ]:
# A new approach for a simple bidirectional a* algorithm
# Simply, it is bidirectional dijkstra using modified edge length l'

def bidirectional_dijkstra_modified_length(g, start, target):
    start_time = time.time()

    # Initialize for implementation
    # open_lst is with priority queue
    open_lst = []
    open_lst_R = []
    closed_lst =[]
    closed_lst_R = []
    hq.heapify(open_lst)
    hq.heapify(open_lst_R)
    
    for i in g.vert_dict.values():
        i.distance = inf
        i.prev = None
        i.distance_R = inf
        i.prev_R = None
        i.visited = False
        i.visited_R = False
    
    start = g.vert_dict[start]
    target = g.vert_dict[target]

    state = 1
    
#   Initialize start & target node
    start.distance = 0
    start.prev = -1
    start.g_score = 0
    target.distance_R = 0
    target.prev_R = -1
    target.g_score_R = 0
    
    closed_lst.append(start)
    closed_lst_R.append(target)
    
    current = start
    current_R = target

    while True:
        # forward search
        state += 1
        for near_vtx, near_vtx_cost in current.adjacent.items():
            if near_vtx.visited == True: continue
            new_distance = current.distance + near_vtx_cost + heuristic(current, near_vtx, start, target)
            if near_vtx.distance > new_distance:
                near_vtx.distance = new_distance
                near_vtx.g_score = current.g_score + near_vtx_cost
                near_vtx.prev = current
                hq.heappush(open_lst, (near_vtx.distance, near_vtx))        

        # By heappop current node is deleted from open list
        nxt_node = hq.heappop(open_lst)
        # Choose the next node
        current = nxt_node[1]
        
        current.visited = True
        closed_lst.append(current)
        if current.visited_R == True: break
        
        # backward search
        state += 1
        for near_vtx, near_vtx_cost in current_R.adjacent_R.items():
            if near_vtx.visited_R == True: continue
            new_distance_R = current_R.distance_R + near_vtx_cost + heuristic(near_vtx, current_R, start, target)
            if near_vtx.distance_R > new_distance_R:
                near_vtx.distance_R = new_distance_R
                near_vtx.g_score_R = current_R.g_score_R + near_vtx_cost
                near_vtx.prev_R = current_R
                hq.heappush(open_lst_R, (near_vtx.distance_R, near_vtx))        

        # By heappop current node is deleted from open list
        nxt_node = hq.heappop(open_lst_R)
        # Choose the next node
        current_R = nxt_node[1]
        
        current_R.visited_R = True
        closed_lst_R.append(current_R)
        if current_R.visited == True: break
            
    # End Phase
    while not len(open_lst) == 0 & len(open_lst_R) == 0:    
        # forward search
        state += 1
        for near_vtx, near_vtx_cost in current.adjacent.items():
            if near_vtx.visited == True: continue
            new_distance = current.distance + near_vtx_cost
            if near_vtx.distance > new_distance:
                near_vtx.distance = new_distance
                near_vtx.g_score = current.g_score + near_vtx_cost
                near_vtx.prev = current

        # By heappop current node is deleted from open list
        if len(open_lst) != 0:
            nxt_node = hq.heappop(open_lst)
            current = nxt_node[1]
            closed_lst.append(current)
        
        # backward search
        state += 1
        for near_vtx, near_vtx_cost in current_R.adjacent_R.items():
            if near_vtx.visited_R == True: continue
            new_distance_R = current_R.distance_R + near_vtx_cost
            if near_vtx.distance_R > new_distance_R:
                near_vtx.distance_R = new_distance_R
                near_vtx.g_score_R = current_R.g_score_R + near_vtx_cost
                near_vtx.prev_R = current_R

        # By heappop current node is deleted from open list
        if len(open_lst_R) != 0 :
            nxt_node = hq.heappop(open_lst_R)
            current_R = nxt_node[1]
            closed_lst_R.append(current_R)
    
    # Select the minimum cost path in set R
    z_lst = list(set(closed_lst)&set(closed_lst_R))
    node_lst = []
    for i in z_lst:
        node_lst.append((i.g_score+i.g_score_R, i))
    
    node = min(node_lst)[1]
    
    # shortest forward path
    path_node = node
    path_f = [path_node.id]    
    while path_node != start:
        path_node = path_node.prev
        path_f.append(path_node.id)
    path_f.reverse()
    
    # shortest backward path
    path_node = node
    path_R = [path_node.id]
    while path_node != target:
        path_node = path_node.prev_R
        path_R.append(path_node.id)

    path = path_f+path_R[1:]
    execution_time = time.time() - start_time
    
    print("[New Approach for a Simple Bidirectional A* algorithm]")
    print(">>> distance cost is:", node.g_score + node.g_score_R)
    print(">>> #iteration: ", state)
    print(">>> #expanded nodes:", len(closed_lst+closed_lst_R))
    print(">>> execution time: ",execution_time)
    
    return path, closed_lst+closed_lst_R

In [ ]:
path, closed_lst = bidirectional_dijkstra_modified_length(graph, 259986397, 9135501698)

In [ ]:
ans = np.array(path)
ans.size